In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from PIL import Image
import wandb
from wandb.keras import WandbCallback

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config =  ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7
session = InteractiveSession(config=config)

In [3]:
batch_size = 32
IMG_SIZE = 224

train_dir = 'D:/dataset/PASCAL_VOC_2007/VOC2007/train'
val_dir = 'D:/dataset/PASCAL_VOC_2007/VOC2007/val'
test_dir = 'D:/dataset/PASCAL_VOC_2007/VOC2007/test'

In [4]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_ds = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE,IMG_SIZE),
    class_mode='categorical',
    batch_size=batch_size
)

val_ds = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE,IMG_SIZE),
    class_mode='categorical',
    batch_size=batch_size
)


test_ds = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE,IMG_SIZE),
    class_mode='categorical',
    batch_size=batch_size
)

Found 5836 images belonging to 20 classes.
Found 1470 images belonging to 20 classes.
Found 7013 images belonging to 20 classes.


In [5]:
train_ds.class_indices

{'aeroplane': 0,
 'bicycle': 1,
 'bird': 2,
 'boat': 3,
 'bottle': 4,
 'bus': 5,
 'car': 6,
 'cat': 7,
 'chair': 8,
 'cow': 9,
 'diningtable': 10,
 'dog': 11,
 'horse': 12,
 'motorbike': 13,
 'person': 14,
 'pottedplant': 15,
 'sheep': 16,
 'sofa': 17,
 'train': 18,
 'tvmonitor': 19}

In [5]:
vgg_model = tf.keras.applications.VGG16(weights = 'imagenet',
                                        include_top = False,
                                        input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [6]:
model = tf.keras.Sequential()

for layer in vgg_model.layers[:-1]: # just exclude last layer from copying
    model.add(layer)
    
for layer in model.layers:
    layer.trainable = False

model.add(tf.keras.layers.GlobalAveragePooling2D())
# model.add(tf.keras.layers.Dense(2048, activation='relu'))
# model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dense(len(train_ds.class_indices), activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [7]:
wandb.init(project="2023 KICS winter", entity="yhkim", config={"learning_rate": 0.001,"epochs": 50})

model.compile(optimizer=tf.optimizers.Adam(learning_rate=wandb.config.learning_rate), 
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=["accuracy", "top_k_accuracy_score"])

history = model.fit(train_ds,
                    epochs=wandb.config.epochs, 
                    steps_per_epoch=len(train_ds),
                    validation_steps=len(val_ds),
                    validation_data=val_ds,
                    callbacks=[WandbCallback()])

wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yhkim (use `wandb login --relogin` to force relogin)
C:\Users\KYH\.conda\envs\tensor\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
wandb: wandb version 0.13.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/50
365/365 [==============================] - 59s 163ms/step - loss: 2.5140 - accuracy: 0.2610 - val_loss: 2.3020 - val_accuracy: 0.3014
Epoch 2/50
365/365 [==============================] - 58s 160ms/step - loss: 2.2176 - accuracy: 0.3204 - val_loss: 2.1178 - val_accuracy: 0.3320
Epoch 3/50
365/365 [==============================] - 57s 157ms/step - loss: 2.0774 - accuracy: 0.3547 - val_loss: 2.0175 - val_accuracy: 0.3633
Epoch 4/50
365/365 [==============================] - 58s 160ms/step - loss: 1.9876 - accuracy: 0.3809 - val_loss: 1.9479 - val_accuracy: 0.3878
Epoch 5/50
365/365 [==============================] - 58s 160ms/step - loss: 1.9316 - accuracy: 0.3974 - val_loss: 1.8961 - val_accuracy: 0.4014
Epoch 6/50
365/365 [==============================] - 59s 161ms/step - loss: 1.8808 - accuracy: 0.4076 - val_loss: 1.8672 - val_accuracy: 0.4129
Epoch 7/50
365/365 [==============================] - 58s 158ms/step - loss: 1.8431 - accuracy: 0.4169 - val_loss: 1.8363 - val_ac

accuracy,▁▃▄▄▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇██████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▆▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▂▄▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇████████████▇███████▇█
val_loss,█▆▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.51765
best_epoch,48
best_val_loss,1.65892
epoch,49
loss,1.4653
val_accuracy,0.46122


In [8]:
model.save('vgg16_voc2007')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: vgg16_voc2007\assets


In [8]:
model = tf.keras.models.load_model('model/resnet50_voc2007')

In [45]:
preds = model.predict(test_ds)

In [9]:
model.evaluate(test_ds)

220/220 [==============================] - 18s 75ms/step - loss: 3.2128 - accuracy: 0.4805 - top_k_categorical_accuracy: 0.8583


[3.212815999984741, 0.4805361330509186, 0.8582632541656494]

In [66]:
y_true = test_ds.labels[test_ds.index_array]

In [67]:
y_true

array([ 0, 14, 14, ...,  4, 14,  6])

In [68]:
np.argmax(preds, axis=1)

array([15,  1, 14, ..., 14, 11, 14], dtype=int64)

In [34]:
from sklearn.metrics import top_k_accuracy_score, accuracy_score

In [72]:
top_k_accuracy_score(y_true, preds, k=5)

0.45387138171966346